<a href="https://colab.research.google.com/github/VipCode786/resumeParser/blob/main/resumeParser2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade en-core-web-sm
!pip install spacy
!pip install spacy_transformers


  Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.0
    Uninstalling spacy-3.7.0:
      Successfully uninstalled spacy-3.7.0


In [ ]:
!python -m spacy validate

2024-01-29 06:43:31.236343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 06:43:31.236395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 06:43:31.238317: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 06:43:31.253662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 06:43:32.457190: W tensorflow/compiler/tf2

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.6.1'

In [ ]:
!nvidia-smi

Mon Jan 29 06:44:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
 cv_data = json.load(open('/content/drive/MyDrive/parser_resume/dataset/dataset1.json','r'))
# import json

# cv_data = []

# # Open the file and read each line
# with open('/content/drive/MyDrive/parser_resume/dataset/dataset.json', 'r') as file:
#     for line in file:
#         try:
#             # Attempt to load the JSON object from the line
#             json_object = json.loads(line)
#             cv_data.append(json_object)
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON: {e}")

# # Now, cv_data contains a list of JSON objects

In [ ]:
len(cv_data)

200

In [ ]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/parser_resume/config/base_config.cfg /content/drive/MyDrive/parser_resume/config/config.cfg

2024-01-29 06:52:32.401878: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 06:52:32.401931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 06:52:32.403137: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 06:52:33.461969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/parser_resume/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spac

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start,end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
        pass

  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)


In [ ]:
len(train), len(test)

(140, 60)

In [ ]:
file = open('/content/drive/MyDrive/parser_resume/model/train_file.txt','w')
db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/parser_resume/model/train_data.spacy')
db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/parser_resume/model/test_data.spacy')
file.close()

# from spacy.tokens import DocBin
# from tqdm.notebook import tqdm

# def get_spacy_doc(file, data):
#     db = DocBin()

#     for item in tqdm(data, desc="Processing data"):
#         if isinstance(item, tuple) and len(item) >= 2:
#             text, annot = item[:2]  # Take the first two elements of the tuple
#             doc = nlp.make_doc(text)
#             annot = annot['entities']
#             example = Example.from_dict(doc, {'entities': annot})
#             db.add(example.reference)

#             # Optionally, write the text to the file
#             file.write(text + "\n")

#     return db

# # Assuming 'train' and 'test' are lists of tuples
# file = open('/content/drive/MyDrive/parser_resume/model/train_file.txt', 'w')
# db_train = get_spacy_doc(file, train)
# db_train.to_disk('/content/drive/MyDrive/parser_resume/model/train_data.spacy')
# file.close()

# file = open('/content/drive/MyDrive/parser_resume/model/test_file.txt', 'w')
# db_test = get_spacy_doc(file, test)
# db_test.to_disk('/content/drive/MyDrive/parser_resume/model/test_data.spacy')
# file.close()


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
!python -m spacy train /content/drive/MyDrive/parser_resume/config/config.cfg  --output /content/drive/MyDrive/parser_resume/model/output  --paths.train /content/drive/MyDrive/parser_resume/model/train_data.spacy  --paths.dev /content/drive/MyDrive/parser_resume/model/test_data.spacy --gpu-id 0

2024-01-29 06:54:03.683307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 06:54:03.683375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 06:54:03.685225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 06:54:05.323816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/parser_resume/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model ch

In [ ]:
 nlp = spacy.load('/content/drive/MyDrive/parser_resume/model/output/model-best')
# nlp = spacy.load('en_core_web_sm')


In [ ]:
!pip install PyMuPdf

In [ ]:
import sys
import fitz
fname = '/content/drive/MyDrive/parser_resume/dataset/Vipul_Gupta_Mern.pdf'

doc = fitz.open(fname)

In [ ]:
# Initialize an empty string to store the extracted text from the PDF
text = " "

# Iterate through each page in the PDF and concatenate the text
for page in doc:
  text = text + str(page.get_text())

# Display the extracted text
# print(text)

In [ ]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)


UI design principles and best practices

Proficiency in HTML, CSS, JavaScript

Making SEO friendly websites

UI frameworks and libraries (e.g., React, Angular)

Backend framework and library (e.g, Node js , Express Js , Mongoose)

SQL and NoSql database ( Sql , Mongodb)

Test case design and documentation

Version control systems (e.g., Git)

Bug tracking tools (e.g., JIRA)

Test planning, execution, and reporting

Defect tracking and management

Software testing methodologies (manual and automated)

Working on Agile Methodologies

Strong problem-solving and analytical skills

Excellent communication and collaboration abilities   ->>>>   Skills
Vipul Gupta   ->>>>   Name
Master of Computer Application   ->>>>   Degree
2021   ->>>>   Graduation Year
Central University   ->>>>   College Name
Bachelor of Computer Application   ->>>>   Degree
DDU Gorakhpur University   ->>>>   College Name


In [ ]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)

# Keep track of entities to avoid repetition
seen_entities = set()

# Iterate through the named entities recognized by the model
for ent in doc.ents:
    # Check if the entity has not been seen before
    if ent.text not in seen_entities:
        # Print the recognized text and its corresponding label
        print(ent.text, "  ->>>>  ", ent.label_)

        # Add the entity to the set to avoid repetition
        seen_entities.add(ent.text)



UI design principles and best practices

Proficiency in HTML, CSS, JavaScript

Making SEO friendly websites

UI frameworks and libraries (e.g., React, Angular)

Backend framework and library (e.g, Node js , Express Js , Mongoose)

SQL and NoSql database ( Sql , Mongodb)

Test case design and documentation

Version control systems (e.g., Git)

Bug tracking tools (e.g., JIRA)

Test planning, execution, and reporting

Defect tracking and management

Software testing methodologies (manual and automated)

Working on Agile Methodologies

Strong problem-solving and analytical skills

Excellent communication and collaboration abilities   ->>>>   Skills
Vipul Gupta   ->>>>   Name
Master of Computer Application   ->>>>   Degree
2021   ->>>>   Graduation Year
Central University   ->>>>   College Name
Bachelor of Computer Application   ->>>>   Degree
DDU Gorakhpur University   ->>>>   College Name
